In [49]:
query = "attendance policy. mandatory classes. mandatory attendance."

In [50]:
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma

In [51]:
import json
from pathlib import Path
from pprint import pprint

In [52]:
file_path='../data/test.json'
json_data = json.loads(Path(file_path).read_text())
print(len(json_data['docs']))


4265


In [53]:
from gpt4all import Embed4All

# Initialize Embed4All
embeddings = Embed4All()
query_vec = embeddings.embed(query)

In [54]:
secondary_data = json.load(open('../data/secondary_data.json'))
print(secondary_data)

{'Judson-MATH-1A-01-W19': {'grading_policy': 'Grade\nYour grade will be computed using the following grade distribution.\n5% Labs 10% Quizzes. 8 Ch. 4.4\n\nQuiz 9\nHw 13 due\n22 Lab 6due\n\n \n\x0cLab Grading Policies. will be graded out of 5 points each. You will not know which 4 problems will be graded. For the. Grading Scale\n\nDue to the complexity of the material the grading scale we will use is as follows. General Grading: Each lab member is required to turn in their own lab report. Failure to turn in a. '}, 'Lien-MATH-1A-05-W19': {'grading_policy': 'Final grades are non-negotiable. You should monitor your scores in the gradebook regularly. QUESTION: What is my current grade in the class?. Breakdown of grades:\nHomework\n63-879% «BO\n\n80-82.9% «|B\n63-679% «(DS\n60-62.9% «(| D-. marks incorrectly. Problems must be brought to my attention immediately after exams are returned.. - You are required to come to class prepared WITH lecture notes printed out.\n\nGRADING. '}, 'Howard-MAT

In [55]:
secondary_data

{'Judson-MATH-1A-01-W19': {'grading_policy': 'Grade\nYour grade will be computed using the following grade distribution.\n5% Labs 10% Quizzes. 8 Ch. 4.4\n\nQuiz 9\nHw 13 due\n22 Lab 6due\n\n \n\x0cLab Grading Policies. will be graded out of 5 points each. You will not know which 4 problems will be graded. For the. Grading Scale\n\nDue to the complexity of the material the grading scale we will use is as follows. General Grading: Each lab member is required to turn in their own lab report. Failure to turn in a. '},
 'Lien-MATH-1A-05-W19': {'grading_policy': 'Final grades are non-negotiable. You should monitor your scores in the gradebook regularly. QUESTION: What is my current grade in the class?. Breakdown of grades:\nHomework\n63-879% «BO\n\n80-82.9% «|B\n63-679% «(DS\n60-62.9% «(| D-. marks incorrectly. Problems must be brought to my attention immediately after exams are returned.. - You are required to come to class prepared WITH lecture notes printed out.\n\nGRADING. '},
 'Howard-M

In [56]:
# tag name cleanup
# for course in secondary_data:
#     if "attendance policy" in secondary_data[course]:
#         secondary_data[course].pop("attendance policy")

In [57]:
# tag name cleanup
# for course in secondary_data:
#     if "grading policy | marks distribution" in secondary_data[course]:
#         content = secondary_data[course]["grading policy | marks distribution"]
#         secondary_data[course].pop("grading policy | marks distribution")
#         secondary_data[course]["grading_policy"] = content

In [47]:
query_tag = "attendance_mandatory"

In [58]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Initialize Embed4All
embeddings = Embed4All()

# Iterate over each document
for doc_id in range(len(json_data['docs'])):
# for doc_id in range(100):
    doc_str = json_data['docs'][doc_id]['content']

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
    doc_split = text_splitter.create_documents([doc_str])

    # Embed the documents
    doc_vecs = np.array([embeddings.embed(sub_doc.page_content) for sub_doc in doc_split])

    # Compute cosine similarity scores
    scores = cosine_similarity([query_vec], doc_vecs)[0]

    # Get top k documents
    k = 5
    top_docs_indices = np.argsort(-scores)[:k]
    top_docs = [doc_split[i] for i in top_docs_indices]

    # print("title:", json_data['docs'][doc_id]['title'])
    # print("matching substrings:")
    final_doc_str = ""
    for sub_doc in top_docs:
        # print(sub_doc.page_content)
        final_doc_str += sub_doc.page_content + ". "
    
    if json_data['docs'][doc_id]['title'] not in secondary_data:
        secondary_data[json_data['docs'][doc_id]['title']] = {}
    secondary_data[json_data['docs'][doc_id]['title']][query_tag] = final_doc_str    
    # secondary_data[json_data['docs'][doc_id]['title']]['retrieved_content'] = final_doc_str

    if doc_id %50 == 0:
        print(doc_id)
    # print("\n")

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250


In [59]:
secondary_data

{'Judson-MATH-1A-01-W19': {'grading_policy': 'Grade\nYour grade will be computed using the following grade distribution.\n5% Labs 10% Quizzes. 8 Ch. 4.4\n\nQuiz 9\nHw 13 due\n22 Lab 6due\n\n \n\x0cLab Grading Policies. will be graded out of 5 points each. You will not know which 4 problems will be graded. For the. Grading Scale\n\nDue to the complexity of the material the grading scale we will use is as follows. General Grading: Each lab member is required to turn in their own lab report. Failure to turn in a. ',
  'attendance_mandatory': 'may, of course, make arrangements with other members of the class to declare yourself as part of. course you are not required to do the Honors Problems. No late work will be accepted. Your. Those of you who need additional accommodations, due to disability, campus-related. Policies later in this document. No late lab assignments will be accepted. Your lowest lab score. extreme circumstances occur it is the students responsibility to inform me immedia

In [60]:
with open('../data/secondary_data.json', 'w') as outfile:
    json.dump(secondary_data, outfile, indent=4)

4265

In [77]:
json_data['docs'][3132]

{'title': 'Hansen-Math-212-61-U16',
 'content': 'Math 212\n\nMATH 212.-61\n\nCourse Goal:\n\nStudent Learning\nObjectives (SLO):\n\nCourse Objectives:\n\nPrerequisite:\n\nTexts:\n\nSupplies:\n\nGreen Sheet—212\n\nPage |\n\nCOLLEGE MATH PREPARATION SUMMER 2016\nLEVEL 2: BEGINNING ALGEBRA\n\nThe goal of this course is to build confidence in students with respect to\ntheir ability to think clearly about a problem, apply mathematical\ntechniques to solve it, and to support the method and solution.\n\nA) Evaluate real-world situations and distinguish between and apply\nLinear and quadratic function models appropriately.\n\nB) Analyze, interpret and communicate results of linear and quadratic\nModels in a logical manner from four points of view — visual,\nformula, numerical and written.\n\nC) Demonstrate an appreciation and awareness of applications in their\ndaily lives.\n\nThe student will:\n\nA) Develop, throughout the course as applicable, systematic problem\nsolving methods\n\nB) Examin

In [78]:
3132 / 4265 * 100

73.43493552168816

## Pick 150 random numbers from 0-3132

In [80]:
import numpy as np

# Set the seed for reproducibility
np.random.seed(42)

# Generate 150 random numbers between 0 and 3132
random_numbers = np.random.randint(0, 3133, size=150)

# Print the array of random numbers
print(random_numbers)


[ 860 1294 1130 1095 3092 1638 2169  466 1238  330 1482 2135 2919  130
 1685  769 2391 1515 2853 2433 1215  955 2324 1184  459   21 2300  747
 2904  474 1082 2558 2047 2747  975 1806  189 3005 2734 3005  562 1899
 1267 2879 1528  646 2068 2888 2214 1297 2435  600 2363 2061  241 2041
 2824 2612 2945 1363 2139 1390 3003 1478 2556  775   34 2253 1955 1585
 3073 1021 2613 1129 1500  702 2449 2777 1579  161  201 1981  995 2317
  815  455 1275 1016 2343 2767  337  878 1076  791 2264  763 2235  379
  492 3112 1180 2062   64 2568 1367 1152 2027 2695 1495 1162 1522 2640
  391 1698  418 2336  378 1796 2278 3065 2088 3099 2182  200 1863  779
 2977 3104 3119  502 2454 1751  804 2146 2731 1895 2773 1570  960 2660
 1485 2690 1028  502 2814  397  870 2842  392  206]


## Generation

In [81]:
import ollama


In [99]:
question="You can only answer in YES or NO. Analyse the following text about a classroom syllabus and respond if student attendance is mandatory or not, in other words do students need to attend all classes or not. If you can't deduce the answer then respond NO. Text : "

In [109]:
result = {}

In [83]:
def find_first_occurrence(s):
    # Initialize positions with a high value
    pos_yes = s.find("yes")
    pos_no = s.find("no")
    # pos_null = s.find("null")
    
    # Filter out substrings not found (pos will be -1)
    positions = [(pos_yes, "yes"), (pos_no, "no")]
    positions = [pos for pos in positions if pos[0] != -1]
    
    if not positions:
        return "no"
    
    # Return the substring with the smallest position
    return min(positions)[1]

In [84]:
secondary_data

{'Judson-MATH-1A-01-W19': {'grading_policy': 'Grade\nYour grade will be computed using the following grade distribution.\n5% Labs 10% Quizzes. 8 Ch. 4.4\n\nQuiz 9\nHw 13 due\n22 Lab 6due\n\n \n\x0cLab Grading Policies. will be graded out of 5 points each. You will not know which 4 problems will be graded. For the. Grading Scale\n\nDue to the complexity of the material the grading scale we will use is as follows. General Grading: Each lab member is required to turn in their own lab report. Failure to turn in a. ',
  'attendance_mandatory': 'may, of course, make arrangements with other members of the class to declare yourself as part of. course you are not required to do the Honors Problems. No late work will be accepted. Your. Those of you who need additional accommodations, due to disability, campus-related. Policies later in this document. No late lab assignments will be accepted. Your lowest lab score. extreme circumstances occur it is the students responsibility to inform me immedia

In [87]:
# Convert the data dictionary to a list of (key, value) pairs
course_list = list(secondary_data.items())

In [98]:
# course_list[0][1]['attendance_mandatory']

'may, of course, make arrangements with other members of the class to declare yourself as part of. course you are not required to do the Honors Problems. No late work will be accepted. Your. Those of you who need additional accommodations, due to disability, campus-related. Policies later in this document. No late lab assignments will be accepted. Your lowest lab score. extreme circumstances occur it is the students responsibility to inform me immediately and. '

In [104]:
course_list[0][0]

'Judson-MATH-1A-01-W19'

In [106]:
cnt = 0

In [110]:
for doc_id in random_numbers:
    course_name = course_list[doc_id][0]
    retrieved_data = course_list[doc_id][1][query_tag]
    prompt = question + "\"" + retrieved_data + "\""
    response = ollama.generate(model='phi3', prompt=prompt)['response'].lower()
        
    if course_name not in result:
        result[course_name] = {}
    result[course_name][query_tag] = find_first_occurrence(response)
    # print(response)
    if cnt % 10 == 0:
        print(cnt)
    cnt += 1

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150


In [111]:
result

{'Howard-Math-22-05-U17': {'attendance_mandatory': 'yes'},
 'Bourgoub-Math-1B-W17': {'attendance_mandatory': 'yes'},
 'Plum-Math-10-31-S17': {'attendance_mandatory': 'yes'},
 'Bloom-Math-1C-09-S17': {'attendance_mandatory': 'yes'},
 'Bloom-Math10-05-U16': {'attendance_mandatory': 'yes'},
 'Khosravi-MATH-2B-03-U20': {'attendance_mandatory': 'no'},
 'Geraghty-MATH-10-04-U18': {'attendance_mandatory': 'yes'},
 'Du-Math-114-65-F15': {'attendance_mandatory': 'yes'},
 'Krestas-MATH-41-21-F19': {'attendance_mandatory': 'yes'},
 'Judson-MATH-114-MP2-S19': {'attendance_mandatory': 'no'},
 'Lien-MATH-1B-05-S20': {'attendance_mandatory': 'no'},
 'Yayli-MATH-210-MP1-S18': {'attendance_mandatory': 'yes'},
 'Dhaliwal-Math-1A-25-S16': {'attendance_mandatory': 'yes'},
 'Rezvani-MATH-114-12-W19': {'attendance_mandatory': 'no'},
 'Judson-MATH-1A-11-W20': {'attendance_mandatory': 'no'},
 'MATH-042-21-S15': {'attendance_mandatory': 'yes'},
 'Mailhot-MATH-1A-21-F18': {'attendance_mandatory': 'yes'},
 'Desi

In [112]:
with open('../data/attendance_llm_result.json', 'w') as outfile:
    json.dump(result, outfile, indent=4)